In [27]:
import boto3
import json
import datetime
from IPython.display import display, JSON

bedrock_agent = boto3.client("bedrock-agent")
bedrock_runtime = boto3.client("bedrock-runtime")
bedrock_agent_runtime = boto3.client("bedrock-agent-runtime")

model_id = "amazon.nova-micro-v1:0"

In [28]:
prompt_name = f"job-description-{datetime.datetime.now().strftime('%Y%m%d%H%M%S')}"

template_text = '''
You are an HR assistant.

Write a professional, inclusive job description using the following inputs:

Job title: {{job_title}}
Responsibilities: {{responsibilities}}
Requirements: {{requirements}}
Location: {{location}}
Work type: {{work_type}}

- Start with a clear summary
- Use concise, inclusive language
- Keep it under 250 words
'''
response = bedrock_agent.create_prompt(
    name=prompt_name,
    description="Generates inclusive job descriptions from structured inputs",
    defaultVariant="v1",
    variants=[
        {
            "name": "v1",
            "modelId": model_id,
            "templateType": "TEXT",
            "templateConfiguration": {
                "text": {
                    "inputVariables": [
                        {"name": "job_title"},
                        {"name": "responsibilities"},
                        {"name": "requirements"},
                        {"name": "location"},
                        {"name": "work_type"}
                    ],
                    "text": template_text
                }
            },
            "inferenceConfiguration": {
                "text": {
                    "maxTokens": 500,
                    "temperature": 0.7,
                    "topP": 0.9,
                    "stopSequences": []
                }
            }
        }
    ]
)
print("\n==================== Response Object ====================\n")
display(JSON(response))

print("\n==================== Prompt ARN ====================\n")
print(response['arn'])
prompt_arn = response['arn']
prompt_id = response['id']


==================== Response Object ====================



<IPython.core.display.JSON object>


==================== Prompt ARN ====================

arn:aws:bedrock:us-east-1:613468529489:prompt/RZIT48Y4TW


In [29]:
response = bedrock_agent.create_prompt_version(
    description='Initial prompt for creating job description documents.',
    promptIdentifier=prompt_id
)

print("\n==================== PROMPT VERSION ARN ====================\n")
print(response["arn"])


==================== PROMPT VERSION ARN ====================

arn:aws:bedrock:us-east-1:613468529489:prompt/RZIT48Y4TW:1


In [30]:
prompt_arn_with_version = prompt_arn

response = bedrock_runtime.converse(
    modelId=prompt_arn_with_version,
    promptVariables={
        "job_title": {"text": "UX Designer"},
        "responsibilities": {"text": "Design user interfaces, run usability testing, collaborate with product teams"},
        "requirements": {"text": "3+ years experience, Figma, HTML/CSS knowledge, communication skills"},
        "location": {"text": "New York or remote"},
        "work_type": {"text": "Full-time"}
    }
)

print("\n==================== Response Text ====================\n")
print(response['output']['message']['content'][0]['text'])


==================== Response Text ====================

**Job Title: UX Designer**

**Summary:**
We are seeking a talented and inclusive UX Designer to join our dynamic team in New York or remotely. As a UX Designer, you will play a pivotal role in designing intuitive user interfaces, conducting usability testing, and collaborating closely with our product teams to deliver exceptional user experiences.

**Responsibilities:**
- Design compelling and user-friendly interfaces that enhance user engagement and satisfaction.
- Conduct thorough usability testing to identify and resolve user experience issues.
- Collaborate with cross-functional product teams to ensure seamless integration of UX insights into the product development lifecycle.

**Requirements:**
- 3+ years of experience in UX design.
- Proficiency in Figma for design prototyping.
- Strong knowledge of HTML/CSS for implementing design specifications.
- Excellent communication skills to effectively convey ideas and collaborate

In [31]:
def handle_response_stream(response):
    try:
        event_stream = response['optimizedPrompt']
        for event in event_stream:
            if 'optimizedPromptEvent' in event:
                print("\n==================== OPTIMIZED PROMPT ====================\n")
                print(event['optimizedPromptEvent']['optimizedPrompt']['textPrompt']['text'])
    except Exception as e:
        raise e


prompt_input = {
    "textPrompt": {
        "text": template_text
    }
}

response = bedrock_agent_runtime.optimize_prompt(
            input=prompt_input,
            targetModelId=model_id
        )

print("\n==================== ORIGINAL PROMPT ====================\n")
print(template_text)
handle_response_stream(response)



==================== ORIGINAL PROMPT ====================


You are an HR assistant.

Write a professional, inclusive job description using the following inputs:

Job title: {{job_title}}
Responsibilities: {{responsibilities}}
Requirements: {{requirements}}
Location: {{location}}
Work type: {{work_type}}

- Start with a clear summary
- Use concise, inclusive language
- Keep it under 250 words


==================== OPTIMIZED PROMPT ====================

"# HR Job Description Generator\n\n## Task\nCreate a professional, inclusive job description based on the provided information.\n\n## Input Information\n### Job Title\n{{job_title}}\n\n### Responsibilities\n{{responsibilities}}\n\n### Requirements\n{{requirements}}\n\n### Location\n{{location}}\n\n### Work Type\n{{work_type}}\n\n## Output Guidelines\n1. Begin with a clear, compelling summary of the position (1-2 sentences)\n2. Format the responsibilities as bullet points, starting with action verbs\n3. Format the requirements as bullet

In [32]:
response = bedrock_agent_runtime.optimize_prompt(
            input=prompt_input,
            targetModelId=model_id
        )

optimized_prompt_text = ""
for event in response['optimizedPrompt']:
    if 'optimizedPromptEvent' in event:
        optimized_prompt_text = event['optimizedPromptEvent']['optimizedPrompt']['textPrompt']['text']
        break

In [33]:
optimized_prompt_text

'"# HR Job Description Generator\\n\\n## Task\\nCreate a professional, inclusive job description based on the provided information.\\n\\n## Inputs\\n### Job Title\\n{{job_title}}\\n\\n### Responsibilities\\n{{responsibilities}}\\n\\n### Requirements\\n{{requirements}}\\n\\n### Location\\n{{location}}\\n\\n### Work Type\\n{{work_type}}\\n\\n## Guidelines\\n1. Begin with a clear, compelling summary of the role and its impact\\n2. Structure the description with clear headings (About the Role, Key Responsibilities, Requirements, What We Offer)\\n3. Use active, gender-neutral, and inclusive language throughout\\n4. Avoid unnecessary jargon or overly complex terminology\\n5. Emphasize both technical requirements and soft skills where applicable\\n6. Highlight flexible work arrangements if available\\n7. Include a brief statement about company commitment to diversity and inclusion\\n8. Ensure the entire description is under 250 words\\n\\n## Output Format\\nProvide a complete, ready-to-publis

In [34]:
prompt_identifier = prompt_arn

existing_prompt = bedrock_agent.get_prompt(promptIdentifier=prompt_identifier)
print("\n==================== ORIGINAL PROMPT ====================\n")
print(existing_prompt["variants"][0]["templateConfiguration"]["text"]["text"])


updated_variants = existing_prompt["variants"]
for variant in updated_variants:
    if variant["templateType"] == "TEXT":
        variant["templateConfiguration"]["text"]["text"] = optimized_prompt_text


response = bedrock_agent.update_prompt(
    promptIdentifier=prompt_identifier,
    name=existing_prompt["name"],
    description=existing_prompt["description"],
    defaultVariant=existing_prompt["defaultVariant"],
    variants=updated_variants
)

print("\n==================== UPDATED PROMPT ====================\n")

updated_prompt = bedrock_agent.get_prompt(promptIdentifier=prompt_identifier)
print(updated_prompt["variants"][0]["templateConfiguration"]["text"]["text"])


==================== ORIGINAL PROMPT ====================


You are an HR assistant.

Write a professional, inclusive job description using the following inputs:

Job title: {{job_title}}
Responsibilities: {{responsibilities}}
Requirements: {{requirements}}
Location: {{location}}
Work type: {{work_type}}

- Start with a clear summary
- Use concise, inclusive language
- Keep it under 250 words


==================== UPDATED PROMPT ====================

"# HR Job Description Generator\n\n## Task\nCreate a professional, inclusive job description based on the provided information.\n\n## Inputs\n### Job Title\n{{job_title}}\n\n### Responsibilities\n{{responsibilities}}\n\n### Requirements\n{{requirements}}\n\n### Location\n{{location}}\n\n### Work Type\n{{work_type}}\n\n## Guidelines\n1. Begin with a clear, compelling summary of the role and its impact\n2. Structure the description with clear headings (About the Role, Key Responsibilities, Requirements, What We Offer)\n3. Use active, gende

In [35]:
response = bedrock_agent.create_prompt_version(
    description='Optimized prompt for creating job description documents.',
    promptIdentifier=prompt_identifier
)

print("\n==================== PROMPT VERSION ARN ====================\n")
print(response["arn"])


==================== PROMPT VERSION ARN ====================

arn:aws:bedrock:us-east-1:613468529489:prompt/RZIT48Y4TW:2


In [36]:
prompt_arn2 = response["arn"]

In [37]:
prompt_arn = prompt_arn

response = bedrock_runtime.converse(
    modelId=prompt_arn,
    promptVariables={
        "job_title": {"text": "UX Designer"},
        "responsibilities": {"text": "Design user interfaces, run usability testing, collaborate with product teams"},
        "requirements": {"text": "3+ years experience, Figma, HTML/CSS knowledge, communication skills"},
        "location": {"text": "New York or remote"},
        "work_type": {"text": "Full-time"}
    }
)

print("\\n==================== Response Text ====================\\n")
print(response['output']['message']['content'][0]['text'])

\n==================== Response Text ====================\n
**Job Title: UX Designer**

**About the Role**  
Join our innovative team as a UX Designer to shape the user experience of our products. You'll design intuitive interfaces, conduct usability tests, and work closely with product teams to ensure our solutions meet user needs.

**Key Responsibilities**  
- Design user interfaces that enhance user satisfaction  
- Conduct usability testing to gather user feedback  
- Collaborate with cross-functional teams to implement design solutions  

**Requirements**  
- 3+ years of UX design experience  
- Proficiency in Figma  
- Knowledge of HTML/CSS  
- Strong communication skills  

**What We Offer**  
- Full-time position  
- Flexible work arrangements: New York or remote  
- Commitment to diversity and inclusion  

We are dedicated to creating a workplace that values diverse perspectives and fosters an inclusive environment. If you're passionate about UX design and want to make a meani

In [38]:
prompt_arn = prompt_arn2

response = bedrock_runtime.converse(
    modelId=prompt_arn,
    promptVariables={
        "job_title": {"text": "UX Designer"},
        "responsibilities": {"text": "Design user interfaces, run usability testing, collaborate with product teams"},
        "requirements": {"text": "3+ years experience, Figma, HTML/CSS knowledge, communication skills"},
        "location": {"text": "New York or remote"},
        "work_type": {"text": "Full-time"}
    }
)

print("\\n==================== Response Text ====================\\n")
print(response['output']['message']['content'][0]['text'])

\n==================== Response Text ====================\n
**Job Title: UX Designer**

**About the Role**  
Join our innovative team as a UX Designer to shape the future of our user interfaces. You'll play a pivotal role in designing intuitive, user-friendly experiences that enhance our product offerings.

**Key Responsibilities**  
- Design compelling user interfaces  
- Conduct usability testing to refine and improve user experiences  
- Collaborate closely with product teams to align designs with business goals  

**Requirements**  
- 3+ years of experience in UX design  
- Proficiency in Figma  
- Strong knowledge of HTML/CSS  
- Excellent communication skills  

**What We Offer**  
- Full-time position  
- Flexible work arrangements: New York or remote  
- A commitment to diversity and inclusion  

We value a diverse workforce and are committed to creating an inclusive environment for all employees. If you're passionate about UX design and ready to make an impact, we'd love to he

In [ ]:
response = bedrock_agent.list_prompts()

for prompt in response['promptSummaries']:
    prompt_id = prompt['id']  # The correct key is 'id', not 'promptId'
    print(f"Deleting prompt: {prompt['name']} (ID: {prompt_id})")
    bedrock_agent.delete_prompt(promptIdentifier=prompt_id)

print("All prompts deleted successfully")

Deleting prompt: job-description-20251111172011 (ID: OFRBF7M6ST)
Deleting prompt: job-description-20251111175903 (ID: RARDXEX6W4)
Deleting prompt: job-description-20251111182049 (ID: SHS4BEADOE)
Deleting prompt: job-description-20251111182112 (ID: SV8TQB29ZY)
Deleting prompt: job-description-20251111194813 (ID: I2H6QMDQSV)
Deleting prompt: job-description-20251111204115 (ID: XMKQT4DX02)
Deleting prompt: job-description-20251111205456 (ID: SW7WOEGKJ5)
Deleting prompt: job-description-20251111205906 (ID: RZIT48Y4TW)
All prompts deleted successfully


: 